In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.datasets import fetch_20newsgroups
data=fetch_20newsgroups()
data.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [21]:
train=fetch_20newsgroups(subset='train',categories=data.target_names)
test=fetch_20newsgroups(subset='test',categories=data.target_names)

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
vector=CountVectorizer()
X_train=vector.fit_transform(train.data)
X_train.shape

(11314, 130107)

In [23]:
from sklearn.feature_extraction.text import TfidfTransformer
vectorizer=TfidfTransformer()
X_train_tfidf=vectorizer.fit_transform(X_train)
X_train_tfidf.shape
X_test=vector.transform(test.data)
X_test_tfidf=vectorizer.transform(X_test)


In [24]:
from sklearn.tree import DecisionTreeClassifier
lr_model= DecisionTreeClassifier()
lr_model=lr_model.fit(X_train_tfidf,train.target)

In [25]:
X_test=vector.transform(test.data)
X_test_tfidf=vectorizer.transform(X_test)
predicted= lr_model.predict(X_test_tfidf)

In [26]:
from sklearn import metrics
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
print("accuracy:",accuracy_score(test.target,predicted))


accuracy: 0.5520446096654275


In [27]:
print("confusion matrix:",test.target,predicted)

confusion matrix: [ 7  5  0 ...  9  6 15] [ 4 12  0 ...  8 12 15]


In [28]:
print(classification_report(test.target,predicted,target_names=test.target_names))

                          precision    recall  f1-score   support

             alt.atheism       0.46      0.47      0.46       319
           comp.graphics       0.44      0.44      0.44       389
 comp.os.ms-windows.misc       0.50      0.59      0.54       394
comp.sys.ibm.pc.hardware       0.41      0.39      0.40       392
   comp.sys.mac.hardware       0.52      0.57      0.55       385
          comp.windows.x       0.48      0.47      0.48       395
            misc.forsale       0.65      0.73      0.69       390
               rec.autos       0.60      0.59      0.59       396
         rec.motorcycles       0.75      0.75      0.75       398
      rec.sport.baseball       0.54      0.55      0.55       397
        rec.sport.hockey       0.68      0.66      0.67       399
               sci.crypt       0.78      0.70      0.74       396
         sci.electronics       0.34      0.34      0.34       393
                 sci.med       0.50      0.44      0.47       396
         